In [1]:
import joblib
import pandas as pd
import requests as r

In [2]:
# load model 1X2
model_results = \
"../data/euro20_results.joblib"
loaded_model = \
joblib.load(model_results)

# load model goals
model_goals = \
"../data/euro20_goals.joblib"
loaded_model_goals = \
joblib.load(model_goals)

# load data
# matches Euro 2024
github = 'https://raw.githubusercontent.com/martj42' 
df_allmatches = pd.read_csv(
f'{github}/international_results/master/results.csv')

# matches Qualifiers Euro 2024
df_qual_24 = pd.read_csv(
    '../data/euro24_qualifiers_norm.csv',
    index_col=0)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [3]:
# get tournament data

headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; \
Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'}
           
all_stats = """
goals,attempts,attempts_on_target,attempts_off_target,
attempts_blocked,matches_appearance,
passes_accuracy,passes_attempted,passes_completed,
ball_possession,cross_accuracy,
cross_attempted,cross_completed,free_kick,
attacks,assists,corners,offsides,
recovered_ball,tackles,clearance_attempted,
saves,goals_conceded,own_goal_conceded,
saves_on_penalty,clean_sheet,
fouls_committed,fouls_suffered,
yellow_cards,red_cards
"""

phase = 'TOURNAMENT'
year = '2024'

URL = f"""
https://compstats.uefa.com/v1/team-ranking?competitionId=3
&limit=60&offset=0&optionalFields=PLAYER,TEAM
&order=DESC&phase={phase}
&seasonYear={year}&stats={all_stats}"""

page = r.get(URL, headers=headers)

# extract the statistics for the teams
stats = page.json()
stats[0]


# extract all data from the json response
def extract_full_data(stats):
    
    full_data = []
    stat_names = []
    for stat in stats:
        team = stat['team']
        all_stats = stat['statistics']    
        data = []
        data.append(team['internationalName'])
        for single_stat in all_stats:
            data.append(float(single_stat['value']))
        full_data.append(data)
    
    stat_names = [all_stats['name'] \
                  for all_stats in stats[0]['statistics']]
    df = pd.DataFrame(full_data, columns=['team_name']+stat_names)
    
    return df 

df_group_24 = extract_full_data(stats)

In [4]:
df_r16 = df_allmatches[
  (df_allmatches.tournament=='UEFA Euro') & 
  (df_allmatches.date>='2024-06-29') & 
  (df_allmatches.date<='2024-07-02')]
len(df_r16)

8

In [5]:
# extract the names of the teams in the round of 16
r16_teams = \
list(df_r16.home_team.unique()) + \
list(df_r16.away_team.unique())
# eliminate duplicates 
r16_teams = list(set(r16_teams))
print(r16_teams)

['Denmark', 'Slovenia', 'Switzerland', 'Slovakia', 'Italy', 'Belgium', 'England', 'Germany', 'Turkey', 'Romania', 'Austria', 'Spain', 'Georgia', 'France', 'Netherlands', 'Portugal']


In [6]:
r16_teams.extend(['Türkiye'])
r16_teams.remove('Turkey')
df_qual_24_filter = df_qual_24[
    (df_qual_24.team_name.isin(r16_teams))]

df_group_24_filter = df_group_24[
    (df_group_24.team_name.isin(r16_teams))]

df_group_24_filter

,team_name,goals,attempts,attempts_on_target,attempts_off_target,attempts_blocked,matches_appearance,passes_accuracy,passes_attempted,passes_completed,...,clearance_attempted,saves,goals_conceded,own_goal_conceded,saves_on_penalty,clean_sheet,fouls_committed,fouls_suffered,yellow_cards,red_cards
0,Germany,8.0,57.0,19.0,21.0,17.0,3.0,93.00,2070.0,1924.0,...,42.0,5.0,2.0,1.0,0.0,1.0,39.0,30.0,5.0,0.0
1,Austria,6.0,31.0,17.0,6.0,8.0,3.0,84.67,1335.0,1136.0,...,47.0,5.0,4.0,1.0,0.0,0.0,49.0,35.0,10.0,0.0
2,Portugal,5.0,54.0,16.0,21.0,17.0,3.0,90.00,1977.0,1783.0,...,33.0,4.0,3.0,0.0,0.0,1.0,25.0,29.0,7.0,0.0
3,Türkiye,5.0,50.0,16.0,17.0,17.0,3.0,88.34,1533.0,1358.0,...,51.0,9.0,5.0,1.0,0.0,0.0,35.0,29.0,16.0,0.0
4,Spain,5.0,48.0,16.0,21.0,11.0,3.0,90.00,1628.0,1470.0,...,40.0,10.0,0.0,0.0,1.0,3.0,46.0,30.0,5.0,0.0
5,Switzerland,5.0,30.0,14.0,10.0,6.0,3.0,83.34,1213.0,1014.0,...,63.0,5.0,3.0,0.0,0.0,0.0,38.0,31.0,8.0,0.0
6,Netherlands,4.0,39.0,10.0,19.0,10.0,3.0,88.00,1406.0,1241.0,...,53.0,12.0,4.0,1.0,0.0,1.0,31.0,33.0,2.0,0.0
7,Romania,4.0,32.0,14.0,13.0,5.0,3.0,77.00,932.0,720.0,...,77.0,13.0,3.0,0.0,0.0,1.0,30.0,27.0,7.0,0.0
8,Georgia,4.0,27.0,8.0,12.0,7.0,3.0,81.67,1021.0,843.0,...,100.0,20.0,4.0,0.0,0.0,1.0,25.0,35.0,6.0,0.0
11,Slovakia,3.0,37.0,13.0,16.0,8.0,3.0,83.00,1352.0,1137.0,...,50.0,12.0,3.0,0.0,0.0,1.0,30.0,34.0,2.0,0.0


In [7]:
# normalize the columns 
for col in df_group_24_filter.columns:
    if col in ['passes_accuracy', 'ball_possession', 
               'cross_accuracy', 'team_name']:
        continue
    else:
        df_group_24_filter[col] = df_group_24_filter[col]/3


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [8]:
df_group_24_filter

,team_name,goals,attempts,attempts_on_target,attempts_off_target,attempts_blocked,matches_appearance,passes_accuracy,passes_attempted,passes_completed,...,clearance_attempted,saves,goals_conceded,own_goal_conceded,saves_on_penalty,clean_sheet,fouls_committed,fouls_suffered,yellow_cards,red_cards
0,Germany,2.666667,19.000000,6.333333,7.000000,5.666667,1.0,93.00,690.000000,641.333333,...,14.000000,1.666667,0.666667,0.333333,0.000000,0.333333,13.000000,10.000000,1.666667,0.0
1,Austria,2.000000,10.333333,5.666667,2.000000,2.666667,1.0,84.67,445.000000,378.666667,...,15.666667,1.666667,1.333333,0.333333,0.000000,0.000000,16.333333,11.666667,3.333333,0.0
2,Portugal,1.666667,18.000000,5.333333,7.000000,5.666667,1.0,90.00,659.000000,594.333333,...,11.000000,1.333333,1.000000,0.000000,0.000000,0.333333,8.333333,9.666667,2.333333,0.0
3,Türkiye,1.666667,16.666667,5.333333,5.666667,5.666667,1.0,88.34,511.000000,452.666667,...,17.000000,3.000000,1.666667,0.333333,0.000000,0.000000,11.666667,9.666667,5.333333,0.0
4,Spain,1.666667,16.000000,5.333333,7.000000,3.666667,1.0,90.00,542.666667,490.000000,...,13.333333,3.333333,0.000000,0.000000,0.333333,1.000000,15.333333,10.000000,1.666667,0.0
5,Switzerland,1.666667,10.000000,4.666667,3.333333,2.000000,1.0,83.34,404.333333,338.000000,...,21.000000,1.666667,1.000000,0.000000,0.000000,0.000000,12.666667,10.333333,2.666667,0.0
6,Netherlands,1.333333,13.000000,3.333333,6.333333,3.333333,1.0,88.00,468.666667,413.666667,...,17.666667,4.000000,1.333333,0.333333,0.000000,0.333333,10.333333,11.000000,0.666667,0.0
7,Romania,1.333333,10.666667,4.666667,4.333333,1.666667,1.0,77.00,310.666667,240.000000,...,25.666667,4.333333,1.000000,0.000000,0.000000,0.333333,10.000000,9.000000,2.333333,0.0
8,Georgia,1.333333,9.000000,2.666667,4.000000,2.333333,1.0,81.67,340.333333,281.000000,...,33.333333,6.666667,1.333333,0.000000,0.000000,0.333333,8.333333,11.666667,2.000000,0.0
11,Slovakia,1.000000,12.333333,4.333333,5.333333,2.666667,1.0,83.00,450.666667,379.000000,...,16.666667,4.000000,1.000000,0.000000,0.000000,0.333333,10.000000,11.333333,0.666667,0.0


In [9]:
# create a combination of the 2 datasets 
# df_group_24_filter + df_qual_24_filter
# loop on teams and create a stat that is an average of the two stats
w_qual = 0.5
w_group = 0.5

all_teams_avg_stats = []
for team in r16_teams:
    # initialize the stats list
    avg_stats = []
    avg_stats.append(team)
    # extract both group and qualifier stats for the team
    group_stat = df_group_24_filter[df_group_24_filter.team_name==team]
    qual_stat = df_qual_24_filter[df_qual_24_filter.team_name==team]
    
    # if Germany then only take group stats into account
    if team == 'Germany':
        for c in group_stat.columns:
            if c == 'team_name':
                continue
            avg_stats.append(group_stat[c].values[0])
            
        all_teams_avg_stats.append(avg_stats)
        continue
        

    for c in group_stat.columns:
        if c == 'team_name':
            continue
        avg_col = w_group*group_stat[c].values[0] + w_qual*qual_stat[c].values[0]
        avg_stats.append(avg_col)
        
    all_teams_avg_stats.append(avg_stats)


In [10]:
df_combine_24 = pd.DataFrame(all_teams_avg_stats,
                             columns=df_group_24_filter.columns)

In [11]:
# replace Türkiye in round-16 matches
df_r16.replace('Turkey', 'Türkiye', inplace=True)
df_r16

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
47348,2024-06-29,Switzerland,Italy,NaN,NaN,UEFA Euro,Berlin,Germany,True
47349,2024-06-29,Germany,Denmark,NaN,NaN,UEFA Euro,Dortmund,Germany,False
47352,2024-06-30,England,Slovakia,NaN,NaN,UEFA Euro,Gelsenkirchen,Germany,True
47353,2024-06-30,Spain,Georgia,NaN,NaN,UEFA Euro,Cologne,Germany,True
47358,2024-07-01,France,Belgium,NaN,NaN,UEFA Euro,Düsseldorf,Germany,True
47359,2024-07-01,Portugal,Slovenia,NaN,NaN,UEFA Euro,Frankfurt,Germany,True
47362,2024-07-02,Romania,Netherlands,NaN,NaN,UEFA Euro,Munich,Germany,True
47363,2024-07-02,Austria,Türkiye,NaN,NaN,UEFA Euro,Leipzig,Germany,True


In [12]:
# create dataset 
# home, away, stats_home, stats_away, result (H, D, A)

all_stats = []

# loop on the df_r16
for idx, row in df_r16.iterrows():
    
    match_stats = []
    home_team = row['home_team']
    away_team = row['away_team']
    
    # get the stats from the combined group and qualifiers  
    home_stats = df_combine_24[df_combine_24['team_name']==home_team].copy()
    away_stats = df_combine_24[df_combine_24['team_name']==away_team].copy()
        
    home_stats = home_stats.add_suffix('_home')
    away_stats = away_stats.add_suffix('_away')
    
    
    match_stats.extend(home_stats.values[0])
    match_stats.extend(away_stats.values[0])
    
    all_stats.append(match_stats)

print(all_stats[0])

['Switzerland', 1.9333333333333336, 13.15, 5.283333333333333, 5.316666666666666, 2.55, 1.0, 86.37, 537.6166666666667, 469.25, 57.205, 20.369999999999997, 17.96666666666667, 3.9833333333333334, 11.366666666666667, 44.25, 1.3, 5.616666666666667, 2.8, 35.28333333333333, 8.75, 16.0, 1.5333333333333332, 1.05, 0.0, 0.0, 0.15, 10.933333333333334, 9.466666666666667, 2.283333333333333, 0.05, 'Italy', 1.5, 13.375, 3.75, 4.979166666666667, 4.645833333333334, 1.0, 88.08500000000001, 609.5625, 539.6875, 55.775000000000006, 30.689999999999998, 18.479166666666664, 5.625, 12.791666666666668, 44.0, 1.2083333333333333, 6.375, 1.5416666666666665, 37.5625, 13.145833333333332, 9.833333333333332, 2.9375, 1.0625, 0.16666666666666666, 0.16666666666666666, 0.1875, 10.25, 12.104166666666668, 2.041666666666667, 0.0]


In [13]:
columns = list(home_stats.columns) + \
list(away_stats.columns) 
df_matches_stats = pd.DataFrame(all_stats, 
                                columns=columns)
df_matches_stats.head()

,team_name_home,goals_home,attempts_home,attempts_on_target_home,attempts_off_target_home,attempts_blocked_home,matches_appearance_home,passes_accuracy_home,passes_attempted_home,passes_completed_home,...,clearance_attempted_away,saves_away,goals_conceded_away,own_goal_conceded_away,saves_on_penalty_away,clean_sheet_away,fouls_committed_away,fouls_suffered_away,yellow_cards_away,red_cards_away
0,Switzerland,1.933333,13.150000,5.283333,5.316667,2.550000,1.0,86.370,537.616667,469.250000,...,9.833333,2.937500,1.062500,0.166667,0.166667,0.187500,10.250000,12.104167,2.041667,0.0000
1,Germany,2.666667,19.000000,6.333333,7.000000,5.666667,1.0,93.000,690.000000,641.333333,...,12.766667,1.383333,0.833333,0.000000,0.000000,0.316667,9.750000,8.850000,1.700000,0.0000
2,England,1.708333,10.916667,4.541667,3.645833,2.729167,1.0,89.875,635.187500,572.416667,...,16.583333,3.550000,0.900000,0.050000,0.000000,0.416667,10.850000,12.116667,1.233333,0.0000
3,Spain,2.395833,16.875000,6.041667,7.000000,3.833333,1.0,90.375,634.770833,575.000000,...,26.016667,5.033333,1.566667,0.100000,0.000000,0.316667,9.716667,10.783333,2.350000,0.0500
4,France,2.145833,18.541667,6.270833,6.583333,5.687500,1.0,89.835,611.479167,551.666667,...,15.062500,3.000000,0.416667,0.062500,0.000000,0.645833,9.479167,9.875000,1.645833,0.0625


In [14]:
X = df_matches_stats.drop(labels=['team_name_home', 
                                  'team_name_away'],
             axis=1)

Y = loaded_model.predict(X)

# decode predictions
predictions = ['X' if pred==0 else str(pred) for pred in Y]

# extract probabilities
Y_proba = loaded_model.predict_proba(X)
max_prob = [max(prob) for prob in Y_proba]

df_matches_stats['prob'] = max_prob

# add back to the matches 
df_matches_stats['result_prediction'] = predictions
# check the slip
df_matches_stats[['team_name_home', 
                  'team_name_away', 
                  'result_prediction',
                  'prob']]

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for 

,team_name_home,team_name_away,result_prediction,prob
0,Switzerland,Italy,1,0.5
1,Germany,Denmark,1,0.6
2,England,Slovakia,X,0.7
3,Spain,Georgia,X,0.7
4,France,Belgium,X,0.5
5,Portugal,Slovenia,X,0.7
6,Romania,Netherlands,X,0.4
7,Austria,Türkiye,2,0.4


In [15]:
X = df_matches_stats.drop(labels=['team_name_home', 
                                  'team_name_away', 
                                  'result_prediction',
                                  'prob'],
             axis=1)

Y = loaded_model_goals.predict(X)

# add back to the matches 
df_matches_stats['goals_prediction'] = Y
# check the slip
df_matches_stats[['team_name_home', 'team_name_away', 
                  'goals_prediction']]

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


,team_name_home,team_name_away,goals_prediction
0,Switzerland,Italy,2.6
1,Germany,Denmark,2.2
2,England,Slovakia,2.7
3,Spain,Georgia,2.0
4,France,Belgium,3.5
5,Portugal,Slovenia,1.0
6,Romania,Netherlands,2.3
7,Austria,Türkiye,2.7
